# Retrain a SSD MobilNet model for object detection with Tensorflow1

This notebook allows you to train a network of neurons for object detection.
You will use Tensorflow 1 to train a quantified MobilNet SSD model (V1 or V2).
The interesting thing about training your model on Tensorflow 1 is that the models offered on *Tensorflow Model Zoo* are already quantified and therefore allow to use the network on TPU.

To use your trained network on TPU you can look at this notebook: link

This training is done on a GPU configuration : 


### File tree : 

tod_tf1  

├── pre_trained   
│	└── <pre_trained-network>   
├── training  
│   └──<project>  
│       ├── <pre_trained-network>  
│       ├── train.record  
│       ├── test.record  
│       ├── pipeline.config 
│       ├── form_detection.pbtxt
│       ├── images   
│           ├── test  
│           │   └── *.jpg, *.png ... *.xml  
│           ├── train  
│           │   └── *.jpg, *.png ... *.xml  
│           └── *.csv  
│       
└── models  
    └── research  
        └── object_detection
    

Download the model *ssd_mobilenet_v2_quantized_coco* on the tensorflow model zoo : https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md
Go to tod_tf1/pre_trained and create a folder name such as the model and put all the file inside


## 1. Installation 

Tcheck that your are in tod_tf1 folder : 

In [25]:
! pwd 

/home/jlc/Tensorflow-Object-Detection-with-TF1.15-forTPU


In [ ]:
! mkdir pre_trained
! cd pre_trained && mkdir ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03
! mkdir training && mkdir form_detection
! cd training/form_detection && mkdir images && mkdir ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03
! cd training/form_detection/images && mkdir train && mkdir test 

#### Installation of Tensorflow models and google coral tutorials : 

In [14]:
! git clone https://github.com/tensorflow/models.git

Clonage dans 'models'...
remote: Enumerating objects: 58799, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (73/73), done.
^Cception d'objets:  33% (19686/58799), 161.32 Mio | 8.46 Mio/s


In [29]:
! cd models && git checkout f788046ca876a8820e05b0b48c1fc2e16b0955bc

Mise à jour des fichiers: 100% (4062/4062), fait.
Note : basculement sur 'f788046ca876a8820e05b0b48c1fc2e16b0955bc'.

Vous êtes dans l'état « HEAD détachée ». Vous pouvez visiter, faire des modifications
expérimentales et les valider. Il vous suffit de faire un autre basculement pour
abandonner les commits que vous faites dans cet état sans impacter les autres branches

Si vous voulez créer une nouvelle branche pour conserver les commits que vous créez,
il vous suffit d'utiliser l'option -c de la commande switch comme ceci :

  git switch -c <nom-de-la-nouvelle-branche>

Ou annuler cette opération avec :

  git switch -

Désactivez ce conseil en renseignant la variable de configuration advice.detachedHead à false

HEAD est maintenant sur f788046c Add model_dir to all tests to avoid "resource not found error". (#6143)


In [34]:
! git clone https://github.com/google-coral/tutorials.git
! cp -r tutorials/docker/object_detection/scripts/* models/research/

In [36]:
! cd /home/jlc/Bureau/tod_tf1&& git clone --recurse-submodules https://github.com/google-coral/pycoral

Clonage dans 'pycoral'...
remote: Enumerating objects: 206, done.
remote: Counting objects: 100% (206/206), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 206 (delta 46), reused 198 (delta 41), pack-reused 0
Réception d'objets: 100% (206/206), 2.88 Mio | 5.11 Mio/s, fait.
Résolution des deltas: 100% (46/46), fait.
Sous-module 'libcoral' (https://github.com/google-coral/libcoral) enregistré pour le chemin 'libcoral'
Sous-module 'libedgetpu' (https://github.com/google-coral/libedgetpu) enregistré pour le chemin 'libedgetpu'
Sous-module 'test_data' (https://github.com/google-coral/test_data) enregistré pour le chemin 'test_data'
Clonage dans '/home/jlc/Bureau/tod_tf1/pycoral/libcoral'...
remote: Enumerating objects: 224, done.        
remote: Counting objects: 100% (224/224), done.        
remote: Compressing objects: 100% (188/188), done.        
remote: Total 224 (delta 34), reused 216 (delta 29), pack-reused 0        
Réception d'objets: 100% (224/224), 8.74 Mio

#### Installation of Cython : 

In [ ]:
sudo apt-get install -y python python-tk
pip install Cython contextlib2 pillow lxml jupyter matplotlib

#### Installation of protoc : 

In [ ]:
! wget https://www.github.com/google/protobuf/releases/download/v3.0.0/protoc-3.0.0-linux-x86_64.zip
! unzip protoc-3.0.0-linux-x86_64.zip -d proto3
! mkdir -p local/bin && mkdir -p local/include
! mv proto3/bin/* local/bin
! mv proto3/include/* local/include
! rm -rf proto3 protoc-3.0.0-linux-x86_64.zip

#### Installation of cocoapi : 

In [ ]:
! git clone --depth 1 https://github.com/cocodataset/cocoapi.git
! (cd cocoapi/PythonAPI && make -j8)
! cp -r cocoapi/PythonAPI/pycocotools/ models/research/
! rm -rf cocoapi

#### Installation of EdgeTPU compiler : 

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
! sudo apt-get update
! sudo apt-get install edgetpu-compiler

PYTHONPATH : 

## 2. Create your dataset 

To retrain a model you have to create your **own dataset** : 
1. Take picture of the object that you have to detect (if you work on the Reachy robot go to this notebook : like)
2. Annotate images with labelImg -> *.xml files
3. Convert XML annotated files to CSV format
4. Convert CSV annotated files to tensorflow record format
5. Create the file label_map.pbtxt which contains the labels of the objects to be recognized.

For all of this step go to https://learn.e.ros4.pro/fr/vision/object_detection_tf2/ section 5. 

## 3. Training

#### Configure the pipeline.config file

The configuration file pipeline.config in the folder pre_trained/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03 has to be copy in the training/form_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03 folder and modifie : 

photo


#### Start the training 

In [ ]:
! cd models/research
! ./retrain_detection_model.sh --num_training_steps 50000 --num_eval_steps 100

## 4.  Export the network from the . pb format to the . tflite format:

In [ ]:
! export ckpt_number=50000
! python object_detection/export_tflite_ssd_graph.py   --pipeline_config_path="${CKPT_DIR}/pipeline.config"   --trained_checkpoint_prefix="${TRAIN_DIR}/model.ckpt-${ckpt_number}"   --output_directory="${OUTPUT_DIR}"   --add_postprocessing_op=true
! tflite_convert   --output_file="${OUTPUT_DIR}/output_tflite_graph.tflite"   --graph_def_file="${OUTPUT_DIR}/tflite_graph.pb"   --inference_type=QUANTIZED_UINT8   --input_arrays="${INPUT_TENSORS}"   --output_arrays="${OUTPUT_TENSORS}"   --mean_values=128   --std_dev_values=128   --input_shapes=1,300,300,3   --change_concat_input_ranges=false   --allow_nudging_weights_to_use_fast_gemm_kernel=true   --allow_custom_ops

## 5. Compile for edgeTPU

In [ ]:
! cd ~/tod_tf1/training/form_detection/ssd_mobilenet_v1_quantized_300x300_coco_2019_01_03/model1/tflite && edgetpu_compiler output_tflite_graph.tflite

## 6. Use your trained network on the TPU : 

I use my network on the Reachy robot with this notebook : link 